In [7]:
! mkdir datasets
! wget https://uncloud.univ-nantes.fr/index.php/s/r6W7oixMM48P59k/download
! mv download datasets/catalogue.tsv

--2019-12-03 20:01:55--  https://uncloud.univ-nantes.fr/index.php/s/r6W7oixMM48P59k/download
Resolving uncloud.univ-nantes.fr (uncloud.univ-nantes.fr)... 193.52.104.60, 2001:660:7220:386:193:52:104:60
Connecting to uncloud.univ-nantes.fr (uncloud.univ-nantes.fr)|193.52.104.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72971322 (70M) [application/octet-stream]
Saving to: ‘download’

download            100%[===================>]  69.59M  10.4MB/s    in 8.8s    

2019-12-03 20:02:06 (7.89 MB/s) - ‘download’ saved [72971322/72971322]



In [0]:
import pandas as pd
import numpy as np

In [11]:
list_parser = lambda x: x[1:-1].split(',')
df = pd.read_csv("datasets/catalogue.tsv",
                        sep="\t",
                        converters={'keywords': list_parser,
                                    "concepts": list_parser})
# This is added in case initial dataset hasn't the right columns names:
df.columns = ['id', 'title', 'language', 'type', 'keywords', 'concepts']
df.head(5)

,id,title,language,type,keywords,concepts
0,1228,"Data, information, design and traffic injuries...",en,mp4,"[design, know, information design, people, ...",['http://en.wikipedia.org/wiki/Information_des...
1,4796,Uncertain Allies,en,pdf,"[north korea, korea, china, north, pyongya...","['http://en.wikipedia.org/wiki/North_Korea', ..."
2,6930,Classification of Web Documents Using a Graph-...,en,mp4,"[subgraph, graph, document, contrast, clas...","['http://en.wikipedia.org/wiki/Hello', 'http:..."
3,7867,Cell and Molecular Neurobiology,en,pdf,"[edition, academic press, molecular, 2nd ed...",['http://en.wikipedia.org/wiki/Massachusetts_I...
4,8160,Advanced Fluid Dynamics of the Environment,en,pdf,"[fluid, eddy viscosity, result velocity, fl...","['http://en.wikipedia.org/wiki/Homework', 'ht..."


In [13]:
def MeanAndMedianArrayLength(df, colname):
    mean = 0
    median_tab = []
    for i in range(0, len(df)):
        add = len(df.loc[i,colname])
        mean += add
        median_tab.append(add)
    mean = mean / len(df)
    print("mean of " + colname + " numbers : " + str(mean))
    median_tab = np.sort(np.array(median_tab))
    middle = int(len(median_tab)/2)
    if len(median_tab) % 2 == 0:
        print("median of " + colname + " numbers : " + str((median_tab[middle-1] + median_tab[middle]) / 2) + "\n")
    else:
        print("median of " + colname + " numbers : " + str(median_tab[middle]) + "\n")

# mean concept and keywords number per document
MeanAndMedianArrayLength(df, "concepts")
MeanAndMedianArrayLength(df, "keywords")

mean of concepts numbers : 10.03375
median of concepts numbers : 10.0

mean of keywords numbers : 19.80065
median of keywords numbers : 20.0



Create Graph

Blank node _x = une ressource
_x :title col.title
_x :id col.id
_x :language col.langage
_x :type col.type
_x :keyword :design
_x :concept :URI_DBPEDIA/ressource

:design rdf:type :keyword

:URI_DBPEDIA/ressource rdf:type :concept

Détecter les ressources qui ont un caractère proche

### **How to request the fuseki's API**

First install the module SPARQLWrapper with anaconda's terminal :
- *Choose the right environnement*
- *Write this piece of code : 'pip install SPARQLWrapper'*
- *Documentation : 'https://rdflib.github.io/sparqlwrapper/'*

Next you need to deploy the Fuseki's server :
- *Go to "/fuseki_server/" folder*
- *Launch the "/fuseki-server.bate*

In [17]:
!pip install SPARQLWrapper

In [0]:
from SPARQLWrapper import SPARQLWrapper, JSON

server_URL = "http://localhost:3030/your_service/sparql"

#sparql = SPARQLWrapper(server_URL)
#sparql.setQuery("""
#    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
#    SELECT ?label
#    WHERE { <http://dbpedia.org/resource/Asturias> rdfs:label ?label }
#""")
#sparql.setReturnFormat(JSON)
#results = sparql.query().convert()

#for result in results["results"]["bindings"]:
    #print(result["label"]["value"])